In [60]:
library(httr)
library(foreign)
        
download_puf <- function(short_puf) {
    puf_base <- "http://meps.ahrq.gov/mepsweb/data_files/pufs/"
    puf_suffix <- "ssp.zip"
    
    zip_filename <- paste0(short_puf, "ssp.zip")
    filename <- paste0(short_puf, ".ssp")
    puf_url <- paste0(puf_base, zip_filename)
    download.file(puf_url, zip_filename)
    
    # unzip
    unzip(zip_filename, files = filename)
    saveName <- paste0(short_puf, ".csv")

    # read sas file and return as csv file
    mydata <- read.xport(filename)
    write.table(mydata, file = saveName, sep = ",")
}

normalize_puf <- function(full_puf) {
    stage_1 <- gsub("C-0", "", full_puf)
    stage_2 <- gsub("C-", "", stage_1)
    
    tolower(stage_2)
}

In [63]:
# Emergency Visit PUF's
# pufs <- c('HC-160E','HC-152E','HC-144E','HC-135E','HC-126E','HC-118E','HC-110E','HC-102E','HC-094E','HC-085E','HC-077E','HC-067E','HC-059E','HC-051E','HC-033E','HC-026E','HC-016E','HC-010E')

# Outpatient Visits
pufs <- c('HC-160F','HC-152F','HC-144F','HC-135F','HC-126F','HC-118F','HC-110F','HC-102F','HC-094F','HC-085F','HC-077F','HC-067F','HC-059F','HC-051F','HC-033F','HC-026F','HC-016F','HC-010F')

puf_downloads = c()
for (puf in pufs) {
    download_puf(normalize_puf(puf))
}

ERROR: Error in lookup.xport(file): file not in SAS transfer format
